In [2]:
import ee
import geemap
import geopandas as gpd
import json

service_account = 'ee-account@airy-galaxy-398310.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '.private-key.json')
ee.Initialize(credentials)

In [3]:
# Define Italy
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Italy = countries.filter(ee.Filter.eq('country_na', 'Italy'))

# Get Corine data
dataset = ee.ImageCollection("COPERNICUS/CORINE/V20/100m")

print('Products:', dataset.aggregate_array('system:index').getInfo())

# print('Bands:', corine2012.bandNames())
image = dataset.filter(ee.Filter.eq('system:index', '2018'))\
    .first()\
    .select('landcover')


Products: ['1990', '2000', '2006', '2012', '2018']


In [4]:
Map = geemap.Map(center=[43.7196, 10.4250], zoom=5)

Map.addLayer(image.clip(Italy), None, 'Landsat Random Clusters')

Map

Map(center=[43.7196, 10.425], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [5]:
print(image.getInfo())
for landclass in image.getInfo()['properties']['landcover_class_names']:
    print(landclass)
print(image.getInfo()['properties'])
print(image.getInfo()['properties']['landcover_class_values'])
print(len(image.getInfo()['properties']['landcover_class_names']))

valid_classes = image.getInfo()['properties']['landcover_class_values']

{'type': 'Image', 'bands': [{'id': 'landcover', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [65000, 46000], 'crs': 'EPSG:3035', 'crs_transform': [100, 0, 900000, 0, -100, 5500000]}], 'version': 1578598451411424, 'id': 'COPERNICUS/CORINE/V20/100m/2018', 'properties': {'system:time_start': 1483228800000, 'landcover_class_names': ['Artificial surfaces; urban fabric; continuous urban fabric', 'Artificial surfaces; urban fabric; discontinuous urban fabric', 'Artificial surfaces; industrial, commercial, and transport units; industrial or commercial units', 'Artificial surfaces; industrial, commercial, and transport units; road and rail networks and associated land', 'Artificial surfaces; industrial, commercial, and transport units; port areas', 'Artificial surfaces; industrial, commercial, and transport units; airports', 'Artificial surfaces; mine, dump, and construction sites; mineral extraction sites', 'Artificial surfaces; mine, dump,

In [6]:
roi = Italy

pixel_data = extract_pixel_data(image, roi, pixel_key='landcover')
print(pixel_data)

NameError: name 'extract_pixel_data' is not defined

In [ ]:
gdf = add_landcover_column(ee_image=image, gpd_file="datasets/mosaic_test.geojson", feature_qnt=524, pixel_key='landcover')

def filter_lists(row):
    str_list = row[1:-1]  # Remove square brackets
    str_list = str_list.split(', ')  # Split by comma and space
    str_list = [float(x) for x in str_list]
    
    return str([item for i, item in enumerate(str_list) if i in valid_classes])

gdf['landcover'] = gdf['landcover'].apply(filter_lists)

gdf.to_file("datasets/new_mosaic_clc.geojson", driver="GeoJSON")